In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from scipy.stats import pearsonr
from collections import Counter

<IPython.core.display.Javascript object>

In [3]:
def convert_to_float(value):
    try:
        # If it's a string enclosed in single quotes, remove quotes and replace comma with dot
        if isinstance(value, str):
            return float(value.replace(",", "."))
        else:
            return float(value)
    except ValueError as e:
        print(e)
        return float("nan")

<IPython.core.display.Javascript object>

In [4]:
def preprocess_numeric_cols(df, columns):
    for col in columns:
        df[col] = df[col].astype(str)  # Ensure string type for string operations
        df[col] = df[col].str.replace("#", "")  # Remove '#' characters
        df[col] = df[col].str.replace(",", ".")  # Replace ',' with '.' for decimals
    return df

<IPython.core.display.Javascript object>

# Data preparation - 209 plant aa data

### Reading the files and extracting relevant information:

In [5]:
plant = "AZ"
xls_files = {}

xls = pd.ExcelFile(
    f"../../../data/raw/209/EMBRAPII hubIC IACC_CIMENTO_{plant}.xlsx",
    engine="openpyxl",
)
xls_files[plant] = xls

<IPython.core.display.Javascript object>

In [6]:
print("Plant: ", plant, xls.sheet_names)

Plant:  AZ ['INSTRUÇÕES', 'Clínquer', '4', '92', '70', '111', '156']


<IPython.core.display.Javascript object>

In [7]:
cement_types_per_plant = []
CEMENT_TYPE_COLS = ["Tipo de cimento", "Classe de resistência"]
dataframes = []

for plant, xls in xls_files.items():
    for sheet_name in xls.sheet_names[2:]:
        df = pd.read_excel(xls_files[plant], sheet_name, header=[1, 2])

        if not df[CEMENT_TYPE_COLS].iloc[2:].isna().all().all():
            cement_type = (
                df[CEMENT_TYPE_COLS]
                .loc[2:]
                .astype(str)
                .sum(axis=1)
                .str.replace(" ", "")
                .str.replace("-", "")
                .str.replace("nan", "")
                .unique()
            )

        else:
            cement_type = (
                df[[("Obs.", "Unnamed: 8_level_1")]]
                .loc[2:]
                .astype(str)
                .sum(axis=1)
                .str.replace(" ", "")
                .str.replace("-", "")
                .unique()
            )

        cement_types_per_plant.append((plant, sheet_name, cement_type))
        dataframes.append(df)

<IPython.core.display.Javascript object>

In [8]:
cement_types_per_plant

[('AZ', '4', array(['CPIIE40'], dtype=object)),
 ('AZ', '92', array(['CPIIIRS32'], dtype=object)),
 ('AZ', '70', array(['CPIND'], dtype=object)),
 ('AZ', '111', array(['CPVARI'], dtype=object)),
 ('AZ', '156', array(['CPIIIRS40'], dtype=object))]

<IPython.core.display.Javascript object>

In [9]:
cements = []
for tup in cement_types_per_plant:
    cements.append(tup[2][0])
Counter(cements)

Counter({'CPIIE40': 1,
         'CPIIIRS32': 1,
         'CPIND': 1,
         'CPVARI': 1,
         'CPIIIRS40': 1})

<IPython.core.display.Javascript object>

In [10]:
dataframes = []

for plant, xls in xls_files.items():
    for sheet_name in xls.sheet_names[2:]:
        df = pd.read_excel(xls_files[plant], sheet_name, header=[0, 1, 2])
        df["Unnamed: 0_level_0", "Unnamed: 0_level_1", "Unnamed: 0_level_2"] = plant
        dataframes.append(df.drop([0, 1], axis=0).reset_index(drop=True).copy())

<IPython.core.display.Javascript object>

In [11]:
df = pd.concat(dataframes, axis=0).reset_index(drop=True)

<IPython.core.display.Javascript object>

## Initial Preprocessing

In [12]:
df = pd.concat(
    [
        df["Unnamed: 0_level_0", "Unnamed: 0_level_1", "Unnamed: 0_level_2"],
        df["Dados iniciais"],
        df["Cimento"]["Análise mineralógica"],
        df["Cimento"]["Análise química"],
        df["Característias físicas do cimento"],
        df["Propriedades do cimento"],
    ],
    axis=1,
)

<IPython.core.display.Javascript object>

In [13]:
# df.info()

<IPython.core.display.Javascript object>

### 1. Dropping columns where either there is no data nor it contributes to the analysis
### 2. Renaming columns to a short identifiable name

In [14]:
COLUMNS_TO_DROP = [
    ("Data", "Medida"),
    ("Horário", "Produção"),
    ("Horário", "Medida"),
    ("Obs.", "Unnamed: 148_level_2"),
    ("Tipo de cimento", "Unnamed: 6_level_2"),
    ("Classe de resistência", "Unnamed: 7_level_2"),
    ("Obs.", "Unnamed: 8_level_2"),
    "Obs.",
    ("Obs.", "Unnamed: 148_level_2"),
    ("Unnamed: 141_level_1", "Obs."),
    # ("Unnamed: 139_level_1", "#400"),
]

COLUMNS_TO_RENAME = {
    ("Unnamed: 0_level_0", "Unnamed: 0_level_1", "Unnamed: 0_level_2"): "Plant",
    ("Cidade", "Unnamed: 1_level_2"): "City",
    ("Data", "Produção"): "Date",  # Arrumar!
    ("Unnamed: 136_level_1", "Blaine"): "Blaine",
    ("Unnamed: 137_level_1", "#200"): "#200",
    ("Unnamed: 138_level_1", "#325"): "#325",
    ("Unnamed: 139_level_1", "#400"): "#400",
    ("Unnamed: 140_level_1", "Massa específica"): "Specific Gravity",
    ("Tempo de pega", "Inicio"): "Initial setting time",
    ("Tempo de pega", "Fim"): "Final setting time",
    ("Resistência à compressão", "1 d"): "CS1",
    ("Resistência à compressão", "3 d"): "CS3",
    ("Resistência à compressão", "7 d"): "CS7",
    ("Resistência à compressão", "28 d"): "CS28",
    "Perda ao fogo": "Loss on Ignition",
    "Resíduo insolúvel": "Insoluble Residue",
    "Sulfato de cálcio": "Calcium sulfate",
    "C3S total": "Total C3S",
    "C2S total": "Total C2S",
    "C2S alpha": "Alpha C2S",
    "C2S beta": "Beta C2S",
    "C2S gama": "Gamma C2S",
    "C4AF": "C4AF",
    "C3A": "C3A",
    "C3A cubic": "Cubic C3A",
    "C3A orto": "Orthorhombic C3A",
    "CaO livre": "Free CaO",
    "Portlandita": "Portlandite",
    "Periclasio": "Periclase",
    "Arcanita": "Arcanite",
    "Aphthalita": "Aphthitalite",
    "Langbeinita": "Langbeinite",
    "Bassanita": "Bassanite",
    "Anidrita": "Anhydrite",
    "Calcita": "Calcite",
    "Dolomita": "Dolomite",
    "Quartzo": "Quartz",
    "Muscovita": "Muscovite",
}

<IPython.core.display.Javascript object>

### Defining a single variable to identify Cement Type

In [15]:
# Single variable for the Cement Type
df["Cement_Type"] = (
    df[
        [
            ("Tipo de cimento", "Unnamed: 6_level_2"),
            ("Classe de resistência", "Unnamed: 7_level_2"),
            ("Obs.", "Unnamed: 8_level_2"),
        ]
    ]
    .fillna("")
    .astype(str)
    .sum(axis=1)
)

# # Preprocessing
df["Cement_Type"] = df["Cement_Type"].replace("Fibro", "Fibrocimento")

# Preprocessing
df["Cement_Type"] = (
    df["Cement_Type"]
    .str.replace("Expedido", "")
    .str.replace("Linha 9", "")
    .str.replace("Itajaí", "")
    .str.replace("D-G", "")
    .str.replace("D-E", "")
    .str.replace("DF2", "")
    .str.replace("CPIIF40", "CP II-F-40")
    .str.replace("CPIIF40", "CP II-F40")
    .str.replace("CP VARI", "CP V-ARI")
    .str.replace("CPVARI", "CP V-ARI")
    .str.replace("CP V-RSARI", "CP V-ARI RS")
    .str.replace("CP III-RS40", "CP III-40 RS")
    .str.replace("CP III-RS32", "CP III-32 RS")
    .str.replace("CP IV-RS32", "CP IV-32 RS")
    .str.replace("CP III40", "CP III-40")
    .str.replace("CP III32", "CP III-32")
    .str.replace("CP I-S40", "CP I-S-40")
    .str.replace("CP I40", "CP I-40")
    .str.replace("CP II-E32", "CP II-E-32")
    .str.replace("CP II-E40", "CP II-E-40")
    .str.replace("CP II-F FIBRO40", "CP II-F-40")
    .str.replace("CP II-F32", "CP II-F-32")
    .str.replace("CP II-F40", "CP II-F-40")
    .str.replace("CP II-Z32", "CP II-Z-32")
    .str.replace("CP II-Z40", "CP II-Z-40")
    .str.replace("CP IV32", "CP IV-32")
    .str.replace("CP IND", "CP I")
    .str.replace("CPINDCC", "CP I")
    .str.strip()
)

df = df.drop(COLUMNS_TO_DROP, axis=1).rename(COLUMNS_TO_RENAME, axis=1).copy()
df = df.drop([0]).reset_index(drop=True)
df = df.infer_objects()

<IPython.core.display.Javascript object>

### Dropping and rename the columns defined above

### Cleaning and converting numeric columns to float

In [16]:
NUMERIC_COLUMNS = [
    "Calcium sulfate",
    "Total C3S",
    "Total C2S",
    "Alpha C2S",
    "Beta C2S",
    "Gamma C2S",
    "C4AF",
    "C3A",
    "Cubic C3A",
    "Orthorhombic C3A",
    "Free CaO",
    "Portlandite",
    "Periclase",
    "Arcanite",
    "Aphthitalite",
    "Langbeinite",
    "Bassanite",
    "Anhydrite",
    "Calcite",
    "Dolimita",
    "Quartz",
    "Muscovite",
    "CaO",
    "MgO",
    "Na2O",
    "Al2O3",
    "SiO2",
    "SO3",
    "K2O",
    "TiO2",
    "Fe2O3",
    "Loss on Ignition",
    "Insoluble Residue",
    "Blaine",
    "#200",
    "#325",
    "#400",
    "Specific Gravity",
    "Final setting time",
    "Initial setting time",
    "CS1",
    "CS3",
    "CS7",
    "CS28",
]

df = preprocess_numeric_cols(df, NUMERIC_COLUMNS)

# Preprocessing to fix numeric columns
df[NUMERIC_COLUMNS] = df[NUMERIC_COLUMNS].map(convert_to_float)

# Converting Date to pandas datetime
df["Date"] = pd.to_datetime(df["Date"])

# Drop rows completely missing
index_to_keep = (
    df.drop(["Plant", "Date", "Cement_Type"], axis=1).dropna(axis=0, how="all").index
)
df = df.iloc[index_to_keep].reset_index(drop=True)

df = df.infer_objects()

could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string 

<IPython.core.display.Javascript object>

## Dropping missing features

### Drop rows completely missing

In [17]:
index_to_keep = (
    df.drop(["Plant", "City", "Date", "Cement_Type"], axis=1)
    .dropna(axis=0, how="all")
    .index
)
df = df.iloc[index_to_keep].reset_index(drop=True)

<IPython.core.display.Javascript object>

### Removing features
 that will be fitted on this data.

1. Removing features with features having ~50% or more of missing values
2. Removing features with zero variance
3. Removing features with more than 70% of zeros

In [18]:
(df.isna().sum() / df.shape[0] * 100).sort_values(ascending=False).to_frame(
    name="Missing (%)"
).style.background_gradient(cmap="Reds")

,Missing (%)
Anhydrite,100.000000
Beta C2S,100.000000
CS1,100.000000
Calcium sulfate,100.000000
Gamma C2S,100.000000
Specific Gravity,100.000000
Alpha C2S,100.000000
Total C3S,100.000000
Total C2S,100.000000
Cubic C3A,15.117417


<IPython.core.display.Javascript object>

<h3>Percentage of missing values:</h3>
<table>
    <th>Plant A - Full Dataset:</th>
    <th></th>
    <tr>
        <td>Anhydrite</td>
        <td>100.00</td>
    </tr>
    <tr>
        <td>Beta C2S</td>
        <td>100.00</td>
    </tr>
    <tr>
        <td>CS1</td>
        <td>100.00</td>
    </tr>  
    <tr>
        <td>Calcium sulfate</td>
        <td>100.00</td>
    </tr>
    <tr>
        <td>Gamma C2S</td>
        <td>100.00</td>
    </tr>
    <tr>
        <td>Specific Gravity</td>
        <td>100.00</td>
    </tr>
    <tr>
        <td>Alpha C2S</td>
        <td>100.00</td>
    </tr>  
    <tr>
        <td>Total C3S</td>
        <td>100.00</td>
    </tr>
    <tr>
        <td>Total C2S</td>
        <td>100.00</td>>
    </tr>
</table>

In [19]:
FEATRUES_TO_DROP = [
    "Anhydrite",
    "Beta C2S",
    "CS1",
    "Calcium sulfate",
    "Gamma C2S",
    "Specific Gravity",
    "Alpha C2S",
    "Total C3S",
    "Total C2S",
]

df = df.drop(labels=FEATRUES_TO_DROP, axis=1)

<IPython.core.display.Javascript object>

In [20]:
df.shape

(2044, 39)

<IPython.core.display.Javascript object>

### Removing features with more than 23% of zeros

In [21]:
zero_values = {}
for col in df.select_dtypes(include="number").columns:
    zero_percentages = (df[df[col].eq(0)].shape[0] / df.shape[0]) * 100
    zero_values[col] = zero_percentages

zero_percentages = pd.Series(zero_values, name=f"Zero (%) for {plant}")
zero_percentages = zero_percentages.sort_values(ascending=False)
zero_percentages = zero_percentages.to_frame(name=f"Zero (%) for {plant}")
zero_percentages.style.background_gradient(cmap="Reds")

,Zero (%) for AZ
Dolimita,95.743640
Arcanite,69.569472
#200,41.585127
Langbeinite,34.246575
Free CaO,15.508806
Aphthitalite,11.007828
TiO2,1.663405
Bassanite,0.587084
Muscovite,0.587084
#325,0.587084


<IPython.core.display.Javascript object>

<h3>Percentage of zero values:</h3>
<table>
    <th>Plant A - Full Dataset:</th>
    <th></th>
    <tr>
        <td>Dolimita</td>
        <td>95.74</td>
    </tr>
    <tr>
        <td>Arcanite</td>
        <td>69.57</td>
    </tr>
    <tr>
        <td>#200</td>
        <td>41.59</td>
    </tr>  
    <tr>
        <td>Langbeinite</td>
        <td>34.25</td>
    </tr>
    <tr>
        <td>Free CaO</td>
        <td>15.51</td>
    </tr>
    <tr>
        <td>Aphthitalite</td>
        <td>11.01</td>
    </tr>
</table>


In [22]:
FEATRUES_TO_DROP = [
    "Dolimita",
    "Arcanite",
    "Langbeinite",
    "#200",
    "Free CaO",
    "Aphthitalite",
]
df = df.drop(labels=FEATRUES_TO_DROP, axis=1)

<IPython.core.display.Javascript object>

### Dropping variable with City and Plant identification

In [23]:
FEATRUES_TO_DROP = ["City", "Plant"]
df = df.drop(labels=FEATRUES_TO_DROP, axis=1)

<IPython.core.display.Javascript object>

### Dropping rows where the target variable is null

In [24]:
df = df[df["CS28"].notna()]

<IPython.core.display.Javascript object>

### Dropping rows where the target variable is zero

In [25]:
df = df[~df["CS28"].eq(0)]

<IPython.core.display.Javascript object>

In [26]:
df.shape

(2044, 31)

<IPython.core.display.Javascript object>

In [27]:
# df.describe().T

<IPython.core.display.Javascript object>

In [28]:
# df_copy = df.copy()

<IPython.core.display.Javascript object>

### Dropping rows where any other variable has a zero value

In [29]:
for col in df.columns:
    df = df[~df[col].eq(0)]

<IPython.core.display.Javascript object>

In [30]:
df.shape

(1966, 31)

<IPython.core.display.Javascript object>

In [31]:
(df.isna().sum() / df.shape[0] * 100).sort_values(ascending=False).to_frame(
    name="Missing (%)"
).style.background_gradient(cmap="Reds")

,Missing (%)
Cubic C3A,13.784334
Orthorhombic C3A,13.784334
TiO2,4.323499
#400,4.272635
Insoluble Residue,3.458800
Loss on Ignition,2.797558
Quartz,1.322482
C3A,1.322482
Bassanite,1.322482
Calcite,1.322482


<IPython.core.display.Javascript object>

## Dropping rows with increased missing values after preprocessing

<h3>Percentage of missing values:</h3>
<table>
</table>

In [32]:
# FEATRUES_TO_DROP = ["C3A", "Aphthitalite"]
# df = df.drop(labels=FEATRUES_TO_DROP, axis=1)

<IPython.core.display.Javascript object>

In [33]:
# (df.isna().sum() / df.shape[0] * 100).sort_values(ascending=False).to_frame(
#     name="Missing (%)"
# ).style.background_gradient(cmap="Reds")

<IPython.core.display.Javascript object>

In [34]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Date,1966,2022-04-12 00:17:34.730417152,2021-01-04 00:00:00,2021-08-19 00:00:00,2022-04-12 00:00:00,2022-12-03 00:00:00,2023-07-17 00:00:00,NaN
C4AF,1946.0,7.413248,1.89,6.0425,8.32,9.11,11.56,2.321772
C3A,1940.0,2.451129,0.34,1.7,2.31,3.21,7.56,1.134442
Cubic C3A,1695.0,2.197917,0.02,1.6,2.06,2.555,7.55,0.927621
Orthorhombic C3A,1695.0,0.494283,0.01,0.01,0.35,0.84,3.24,0.504097
Portlandite,1946.0,1.102852,0.03,0.67,1.035,1.42,4.29,0.599619
Periclase,1946.0,0.442754,0.02,0.31,0.44,0.56,1.52,0.185082
Bassanite,1940.0,1.295526,0.03,0.93,1.26,1.63,3.56,0.53886
Calcite,1940.0,5.930041,0.42,4.48,5.26,7.3025,13.3,1.978259
Quartz,1940.0,1.044201,0.02,0.7,0.93,1.26,6.58,0.52976


<IPython.core.display.Javascript object>

In [35]:
df.describe(exclude="number").T

,count,unique,top,freq,mean,min,25%,50%,75%,max
Date,1966,NaN,NaN,NaN,2022-04-12 00:17:34.730417152,2021-01-04 00:00:00,2021-08-19 00:00:00,2022-04-12 00:00:00,2022-12-03 00:00:00,2023-07-17 00:00:00
Cement_Type,1966,5,CP I,635,NaN,NaN,NaN,NaN,NaN,NaN


<IPython.core.display.Javascript object>

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1966 entries, 0 to 2043
Data columns (total 31 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  1966 non-null   datetime64[ns]
 1   C4AF                  1946 non-null   float64       
 2   C3A                   1940 non-null   float64       
 3   Cubic C3A             1695 non-null   float64       
 4   Orthorhombic C3A      1695 non-null   float64       
 5   Portlandite           1946 non-null   float64       
 6   Periclase             1946 non-null   float64       
 7   Bassanite             1940 non-null   float64       
 8   Calcite               1940 non-null   float64       
 9   Quartz                1940 non-null   float64       
 10  Muscovite             1940 non-null   float64       
 11  CaO                   1961 non-null   float64       
 12  MgO                   1965 non-null   float64       
 13  Na2O                  1

<IPython.core.display.Javascript object>

### Sort the dataset by date

In [37]:
# We first make sure Date column was infered correctly
df["Date"].dtype

dtype('<M8[ns]')

<IPython.core.display.Javascript object>

In [38]:
df = df.sort_values(by="Date")

<IPython.core.display.Javascript object>

## Saving the dataset

<h3>Saving for feature engineering</h3>

In [39]:
df.to_csv("../../../data/interim/209/az.csv", index=False)

<IPython.core.display.Javascript object>